In [ ]:
# @title Initial setup

# Install diffusers and fix the version
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!git checkout c4d282360184686f7d4a66787a4be9898cf1b8b0
!pip install -e .

# Install LoRa for Dreambooth requirements
%cd examples/dreambooth/
!pip install -r requirements.txt
%cd ../../..

# Get the data
!git clone https://github.com/axchizhov/kabina_spyachky

# Misc
!pip install wandb

# Setup
import wandb
import accelerate


accelerate.utils.write_basic_config()


# TODO: set_seed()

In [ ]:
# @title Set the inputs
model_name = "runwayml/stable-diffusion-v1-5" # @param {type:'string'}
output_dir = "output" # @param {type:'string'}
reference_subject_dir = "/content/kabina_spyachky/Mathilda/from_movie_many" # @param {type:'string'}


# @markdown #### Wandb monitoring:
enable_monitoring = True # @param {type:"boolean"}
wandb_project_name = "kabina" # @param {type:'string'}
wandb.init(project=wandb_project_name) if enable_monitoring else wandb.init(mode="disabled")

In [ ]:
# @title Run the experiment

script = "diffusers/examples/dreambooth/train_dreambooth_lora.py"

!accelerate launch $script \
  --pretrained_model_name_or_path=$model_name  \
  --instance_data_dir=$reference_subject_dir \
  --output_dir=$output_dir \
  --instance_prompt="a photo of sks girl" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=300 \
  --validation_prompt="A photo of sks girl with a gun" \
  --validation_epochs=50 \
  --seed="0" \
  --report_to="wandb"

wandb.finish()

In [ ]:
# TODO: upload weights to wandb